**Realizado por**: José Javier Díaz González

**Correo**: alu0101128894@ull.edu.es

Esta práctica ha sido mi primera toma de contacto con Python y el código de diferentes librería y modelos matemáticos de problemas de logística.

Como es un primer paso, he querido añadir a esta tarea los pasos introductorios del cuaderno del profesor, para demostrar que los he realizado. Sin embargo, no han sido modificados.

En el apartado de 'Problemas de Asignación' se encuentra el código del cuaderno, con las puntualizaciones, modificaciones y conclusiones que he ido generando a lo largo de su ejecución y comprensión.

# Introducción

Principalmente aprendemos a representar problemas reales mediante modelos matemáticos de programación lineal (continua o entera), es decir, del tipo $\max\{ c^Tx: Ax\leq b, x\in Z^n\}$. Luego estos modelos matemáticos serán resueltos mediante herramientas informáticas especializadas, como [Cplex](https://www.ibm.com/analytics/cplex-optimizer), [Xpress](http://www.fico.com/es/products/fico-xpress-optimization), Gurobi, SCIP, GLKP, CBC, GLOP, etc. Aunque dedicamos algo a comprender las metodologías implementadas en estas herramientas, principalmente la asignatura insistirá en la modelización de problemas.

Un ejemplo de modelo matemático es:
\begin{eqnarray}
\max\;\; x+10y& & \\
x+7y&\leq& 17.5  \\
x &\leq& 3.5  \\
x,y&\in& Z
\end{eqnarray}

## Lenguaje PuLP de COIN-OR

A fín de no complicarnos con la instalación de licencias comerciales, comenzamos usando una librería de optimización de software libre que permite ser llamada desde Python y que ha sideo desarrollada en el proyecto [COIN-OR](https://www.coin-or.org/). Se llama CBC. Además en este mismo proyecto existe un lenguaje que amplía el Python y se llama [PuLP](https://www.coin-or.org/PuLP/). Usando este lenguaje el modelo anterior se resuelve mediante:


In [5]:
!pip install pulp
from pulp import *


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
prob = LpProblem("MiPrimerProblemaEntero", LpMaximize)
X = LpVariable("X", 0, None,cat='Integer')
Y = LpVariable("Y", 0, None,cat='Integer')
prob += X + 10 * Y
prob += X + 7 * Y <= 17.5
prob += X <= 3.5

In [7]:
prob.solve()

print("Status = ", LpStatus[prob.status])                    
print("Optimal objective value = ", value(prob.objective))
print("  X = ", X.varValue)
print("  Y = ", Y.varValue)
# prob.writeLP("modelo.lp")
# print(prob)

Status =  Optimal
Optimal objective value =  23.0
  X =  3.0
  Y =  2.0


Pero con el lenguaje PuLP podemos usar otras librerias de optimización.  Por ejemplo podemos usar GLPK o LPSOLVER que son otras librerías gratuitas de optimización, o Gurobi o Cplex o Xpress que son de pago:

In [6]:
!pip install gurobipy

ErrorException: syntax: extra token "install" after end of expression

In [ ]:
from gurobipy import *

prob.solve( GUROBI( TimeLimit=60 , msg=False ) ) # Error GUROBI No disponible

print("Status = ", LpStatus[prob.status])
print("Optimal objective value = ", value(prob.objective))
print("  X = ", X.varValue)
print("  Y = ", Y.varValue)

In [ ]:
!pip install cplex
from cplex import *

prob.solve( CPLEX(msg=False) )                  # Error PuLP: cannot execute cplex

print("Status = ", LpStatus[prob.status])
print("Optimal objective value = ", value(prob.objective))
print("  X = ", X.varValue)
print("  Y = ", Y.varValue)

In [ ]:
!pip install xpress
from xpress import *

prob.solve( XPRESS(msg=True, keepFiles=True) )     # PuLP: cannot execute optimizer

print("Status = ", LpStatus[prob.status])
print("Optimal objective value = ", value(prob.objective))
print("  X = ", X.varValue)
print("  Y = ", Y.varValue)

Lo positivo es que este lenguaje permite usar fácilmente muchas librerías de optimización (comerciales o gratuitas). Lo negativo es que no permite funciones avanzadas de ninguna librería, sino sólo las funciones genéricas comunes a todas.

## Lenguaje de Google OR-tools

Otro lenguaje alternativo es que ofrece Google, llamado [OR-tools](https://developers.google.com/optimization/). Ni PuLP ni OR-tools son librerías de optimización, sino extensiones del lenguaje Python para escribir cómodamente modelos matemáticos de Programación Lineal (Entera o Continua) que luego pueden ser resueltas con cualquiera de las librerias citadas (CBC, Cplex, Gurobi,...). Actúan como puente entre quien desea resolver un modelo y quien lo resuelve. Muy cómodo. Muy recomendable. 

Para usar la herramienta OR-tools de Google:

In [51]:
!pip install ortools
from ortools.linear_solver import pywraplp


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Via este lenguaje se pueden usar varias librerías de optimización (comerciales y gratuitas) pero hay especialmente 2 librerías gratuitas de optimización muy simples de usar. Uno para programación lineal **continua** llamado GLOP_LINEAR_PROGRAMMING que está construido por Google, y otro para programación lineal **entera** llamado CBC_MIXED_INTEGER_PROGRAMMING que está prestado por Coin-OR. Existe la posibilidad de usar otras librerías de optimización (terceros), pero para ello hay que instalarse el código fuente de Google OR-tools en tu ordenador local.

In [52]:
solver = pywraplp.Solver('MiPrimerProblemaEntero', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
#solver = pywraplp.Solver('MiPrimerProblemaEntero', pywraplp.Solver.SCIP_MIXED_INTEGER_PROGRAMMING)
#solver = pywraplp.Solver('MiPrimerProblemaEntero', pywraplp.Solver.GLPK_MIXED_INTEGER_PROGRAMMING)
#solver = pywraplp.Solver('MiPrimerProblemaEntero', pywraplp.Solver.CPLEX_MIXED_INTEGER_PROGRAMMING)
#solver = pywraplp.Solver('MiPrimerProblemaEntero', pywraplp.Solver.GUROBI_MIXED_INTEGER_PROGRAMMING)
#solver = pywraplp.Solver('MiPrimerProblemaEntero', pywraplp.Solver.XPRESS_MIXED_INTEGER_PROGRAMMING)
#solver = pywraplp.Solver('MiPrimerProblemaEntero', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)  # solo continua
#solver = pywraplp.Solver('MiPrimerProblemaEntero', pywraplp.Solver.BOP_INTEGER_PROGRAMMING)  # solo 0-1

x = solver.IntVar(0.0, solver.infinity(), 'x')
y = solver.IntVar(0.0, solver.infinity(), 'y')

solver.Maximize(x + 10 * y)
solver.Add( x + 7 * y <= 17.5 )
solver.Add( x <= 7.5 )  

result_status = solver.Solve()
assert result_status == pywraplp.Solver.OPTIMAL
  
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())
print('Optimal objective value = %d' % solver.Objective().Value())
  
variable_list = [x, y]
for variable in variable_list:
    print('   %s = %d' % (variable.name(), variable.solution_value()))

Number of variables = 2
Number of constraints = 2
Optimal objective value = 23
   x = 3
   y = 2


Tenemos en OR-tools dos modos de escribir los detalles del modelo: o bien coeficiente a coeficiente, o bien fila a fila. Arriba hemos usado el fila-a-file y a abajo verás el columna-a-columna.

In [53]:
solver = pywraplp.Solver('MiPrimerProblemaEntero', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# x and y are integer non-negative variables.
x = solver.IntVar(0.0, solver.infinity(), 'x')
y = solver.IntVar(0.0, solver.infinity(), 'y')

# x + 7 * y <= 17.5
constraint1 = solver.Constraint(-solver.infinity(), 17.5)
constraint1.SetCoefficient(x, 1)
constraint1.SetCoefficient(y, 7)

# x <= 3.5
constraint2 = solver.Constraint(-solver.infinity(), 3.5)
constraint2.SetCoefficient(x, 1)
constraint2.SetCoefficient(y, 0)

# Maximize x + 10 * y.
objective = solver.Objective()
objective.SetCoefficient(x, 1)
objective.SetCoefficient(y, 10)
objective.SetMaximization()

"""Solve the problem and print the solution."""
result_status = solver.Solve()
# The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL
  
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())
print('Optimal objective value = %d' % solver.Objective().Value())
  
# The value of each variable in the solution.
variable_list = [x, y]
for variable in variable_list:
    print('   %s = %d' % (variable.name(), variable.solution_value()))

Number of variables = 2
Number of constraints = 2
Optimal objective value = 23
   x = 3
   y = 2


Personalmente creo que el modo "fila a fila" es más claro. Pero pueden haber usuarios que prefieran el "columna a columna" (que fue el formato original de las primeras herramientas informáticas en Programación Matemática).

El anterior ejemplo fue un modelo ejemplo muy pequeño, con tan sólo 2 variables y 2 restricciones. Es un ejemplo numérico diminuto, de nulo interés práctico. En la práctica se plantean problemas más complejos que necesitan ser representado mediante modelos con más variables y restricciones, y en consecuencia donde notación abreviada ayuda a entenderlos mejor y reducir la posibilidad de cometer errores al introducirlos en un ordenador. Veamos un ejemplo de problema mayor en la siguiente sección.

## Gratis frente a Comercial

Los modelos de programación entera pequeños pueden ser muy difíciles. Aqui lo mostramos con 3 variables y 1 ecuación:

In [54]:
solver = pywraplp.Solver('ProblemaEnteroDificil',  pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING )

x = solver.IntVar(0.0, solver.infinity(), 'x')
y = solver.IntVar(0.0, solver.infinity(), 'y')
z = solver.IntVar(0.0, solver.infinity(), 'z')

solver.SetTimeLimit( 60 * 1000 )   # le ponemos un tiempo límite de 60 segundos para no calentar el ordenador

solver.Minimize(x)
solver.Add( 75001 * y + 75002 * z == 75000 + 75000 * x )
result_status = solver.Solve()

if result_status == pywraplp.Solver.OPTIMAL:
  
    print('Number of variables =', solver.NumVariables())
    print('Number of constraints =', solver.NumConstraints())
    print('Optimal objective value = %d' % solver.Objective().Value())
  
    variable_list = [x, y, z]
    for variable in variable_list:
         print('   %s = %d' % (variable.name(), variable.solution_value()))
else:
    print(' Finalizó sin optimo : status = ',result_status)
    
print("Time = ", solver.WallTime(), " milliseconds")
print("Iterations = ", solver.Iterations())
print("Nodes = ", solver.nodes())

 Finalizó sin optimo : status =  6
Time =  60524  milliseconds
Iterations =  1239921
Nodes =  1856615


Mismo problema con Gurobi, aunque no te funcionará si no tienes la licencia apropiada en el ordenador donde lo quieras resolver:

In [55]:
from gurobipy import *

modelo = Model("ProblemaEnteroDificil")

x = modelo.addVar(vtype=GRB.INTEGER, name="x")
y = modelo.addVar(vtype=GRB.INTEGER, name="y")
z = modelo.addVar(vtype=GRB.INTEGER, name="z")

modelo.setObjective(x, GRB.MINIMIZE)
modelo.addConstr( 75001 * y + 75002 * z == 75000 + 75000 * x , "c0")
modelo.optimize()

print('Optimal  objective value = %g' % modelo.objVal)
for variable in modelo.getVars():
    print('  %s = %g' % (variable.varName, variable.x))
print('runtime is ',modelo.Runtime)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1 rows, 3 columns and 3 nonzeros
Model fingerprint: 0xe72ebd3d
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [8e+04, 8e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+04, 8e+04]
Presolve removed 1 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 37500 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.750000000000e+04, best bound 3.750000000000e+04, gap 0.0000%
Optimal  objective value = 37500
  x = 37500
  y = 0
  z = 37500
runtime is  0.026000022888183594


Si comparas los tiempos de cálculo de esas dos herramientas entenderás mejor la diferencia entre "gratis" (CBC) y "de pago" (Gurobi)!!!

Comencemos ahora con problemas más serios, de más utilidad.

# Problemas de Asignación

Imaginemos tener 6 trabajadores y 4 tareas, y conocer el costo de $c_{ij}$ de que el trabajador $i$ esté asignado a la tarea $j$. Se busca asignar un trabajador a cada tarea, y que ningún trabajador esté asignado a más de una máquina, de manera que el coste total sea lo menor posible.


In [56]:
# Matriz de costes. Podemos generar costes aleatorios y de manera automática cambiando
# el valor individual de cada elemento por la expresión: 'random.randint([valor mínimo],[valor máximo])'
import random

cost  =  [[ random.randint(1,100),  random.randint(1,100),  random.randint(1,100),  random.randint(1,100)],
          [ random.randint(1,100),  random.randint(1,100),  random.randint(1,100),  random.randint(1,100)],
          [ random.randint(1,100),  random.randint(1,100),  random.randint(1,100),  random.randint(1,100)],
          [ random.randint(1,100),  random.randint(1,100),  random.randint(1,100),  random.randint(1,100)],
          [ random.randint(1,100),  random.randint(1,100),  random.randint(1,100),  random.randint(1,100)],
          [ random.randint(1,100),  random.randint(1,100),  random.randint(1,100),  random.randint(1,100)]]  

#cost  =  [[ 90,  76,  75,  70],
#          [ 35,  85,  55,  65],
#          [125,  95,  90, 105],
#          [ 45, 110,  95, 115],
#          [ 60, 105,  80,  75],
#          [ 45,  65, 110,  95]]     # Matriz de coste de cada tarea por cada trabajador
workers = range( len(cost) )        # Array de número de trabajadores => len(cost) significa la longitud (nº de elementos) en el array: 6 arrays
tasks   = range( len(cost[1]) )     # Array de número de tareas => len(cost[1]) significa la longitud (nº de elementos) en un array dentro de un array: 4 elementos
EPS     = 0.0001

In [57]:
!pip install ortools


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
from ortools.linear_solver import pywraplp
# Inicialización de la variable donde almacenar la solución.
solver = pywraplp.Solver('ProblemaAsignacion2dimensional', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)   

x = { (i,j) : solver.BoolVar('x[%i, %i]' % (i, j)) for i in workers for j in tasks }

# Función Objetivo: minimar el sumatorio de los costes de cada tarea realizada por cada trabajador.
solver.Minimize(  solver.Sum(   cost[i][j] * x[i,j]      for i in workers   for j in tasks   )   )
[ solver.Add(solver.Sum(x[i,j] for j in tasks) <= 1)  for i in workers ]    # Restricción de que cada Trabajador no puede realizar más de una tarea.
[ solver.Add(solver.Sum(x[i,j] for i in workers) >= 1)  for j in tasks ]    # Restricción de que cada Tarea debe poder ser realizada por al menos 1 trabajador. 
                                                                            # (De estos candidatos se elige el más óptimo después)

solver.Solve()        # Resolvemos
# print('x[%i, %i]' % (x[1,1]))   # Intento de imprimir el valor X que son 
print('Costo total = ', solver.Objective().Value() ," in ", solver.WallTime()/1000, "seconds")
for i in workers:
    for j in tasks:
        if x[i, j].solution_value() > EPS :   # Si el valor de la solución, es decir, el costo de realiza la tarea es mayor 0.0001
                                              # significa que la tarea óptima es realizada.
            print('Trabajador %d asignado a la tarea %d.  Costo = %d' % (i,j,cost[i][j]))

Costo total =  67.0  in  0.005 seconds
Trabajador 1 asignado a la tarea 3.  Costo = 22
Trabajador 2 asignado a la tarea 2.  Costo = 11
Trabajador 4 asignado a la tarea 1.  Costo = 32
Trabajador 5 asignado a la tarea 0.  Costo = 2


Si los trabajadores estuviesen en dos equipos (por ejemplo, pares e impares) la solución óptima obtenida usa más trabajadores de un equipo que del otro. Podría tener sentido que el problema real también desee una solución donde se elijan exactamente 2 trabajadores de cada equipo. Para ello hay que introducir nuevas restricciones en el modelo matemático:

In [59]:
team1 = [0, 2, 4]
team2 = [1, 3, 5]

solver.Add(solver.Sum(x[i, j] for i in team1 for j in tasks) <= 2)    # Restriccion de que por cada tarea como máximo la realicen 2 integrantes de un equipo.
solver.Add(solver.Sum(x[i, j] for i in team2 for j in tasks) <= 2)    # Misma restricción para el equipo 2.

solver.Solve()

print('Costo total = ', solver.Objective().Value() ," in ", solver.WallTime()/1000, "seconds")
for i in workers:
    for j in tasks:
        if x[i, j].solution_value() > 0:                              # Solo hemos generado la solución óptima
            print('Trabajador %d asignado a la tarea %d.  Costo = %d' % (i,j,cost[i][j]))

Costo total =  67.0  in  5.491 seconds
Trabajador 1 asignado a la tarea 3.  Costo = 22
Trabajador 2 asignado a la tarea 2.  Costo = 11
Trabajador 4 asignado a la tarea 1.  Costo = 32
Trabajador 5 asignado a la tarea 0.  Costo = 2


Como era de esperar, la nueva solución óptima cuesta más que la anterior porque debe cumplir además las nuevas desigualdades.

Costo total individual = 235 Costo total por equipos = 250

## 3-dimensional

Terminemos esta sección con un problema de asignación más complejo, conocido como **problema de asignación 3-dimensional**. Pensemos ahora que tenemos 5 profesores, 5 asignaturas y 5 aulas, y queremos asignar un profesor a cada asignatura en un aula, de modo que cada profesor da 1 asignatura en 1 aula, cada asignatura la da 1 profesor en 1 aula, y cada aula imparte 1 asignatura por 1 profesor. Todo queremos que se realice a costo total mínimo. Concretamente se llama problema de asignación 3-dimensional **axial** porque como veremos su modelo matemático se basa en restricciones que tienen 1 índice libre, en contraposición a otro problema que se llama de asignación 3-dimensional **planar** donde su modelo se basa en restricciones que tienen 2 indices libres. Vemos primero el modelo para el problema axial planteado, es decir, asignar profesores, asignaturas y aulas:

In [60]:
import random  
dimension = 10
opciones  = range(dimension)       
#Podemos cambiar la semilla para que genere problemas nuevos
#random.seed(12345)
random.seed(54321)  
cost      = [[[random.randint(1,100) for i in opciones] for j in opciones] for k in opciones] # Generación aleatoria de costes
EPS       = 0.001

In [61]:
solver = pywraplp.Solver('3APaxial', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

x = { (i,j,k) : solver.BoolVar('x[%i,%i,%i]' % (i,j,k)) for i in opciones for j in opciones for k in opciones }

# Función objetivo: 
        
solver.Minimize( solver.Sum(cost[i][j][k] * x[i,j,k] for i in opciones for j in opciones for k in opciones))

# Intento de añadir una restricción fija al modelo:
# La restricción consiste en imponer que: 
# el Profesor 0 imparta la Asignatura 0 en el Aula 0, siempre. 
#[ solver.Add(x[0,0,0] = 8)]

[ solver.Add(solver.Sum(x[i,j,k] for j in opciones for k in opciones) == 1) for i in opciones ]
[ solver.Add(solver.Sum(x[i,j,k] for i in opciones for k in opciones) == 1) for j in opciones ]
[ solver.Add(solver.Sum(x[i,j,k] for i in opciones for j in opciones) == 1) for k in opciones ]
    
sol = solver.Solve()

print('Costo total = ', solver.Objective().Value() ," in ", solver.WallTime()/1000, "seconds")
for i in opciones:
    for j in opciones:
        for k in opciones:
            if x[i,j,k].solution_value() > EPS :
                print('Prof %d con asignatura %d en aula %d.  Costo = %d' % (i,j,k,cost[i][j][k]))

Costo total =  37.0  in  0.117 seconds
Prof 0 con asignatura 3 en aula 8.  Costo = 6
Prof 1 con asignatura 9 en aula 1.  Costo = 2
Prof 2 con asignatura 4 en aula 4.  Costo = 3
Prof 3 con asignatura 7 en aula 0.  Costo = 4
Prof 4 con asignatura 1 en aula 9.  Costo = 1
Prof 5 con asignatura 2 en aula 3.  Costo = 4
Prof 6 con asignatura 0 en aula 7.  Costo = 5
Prof 7 con asignatura 5 en aula 2.  Costo = 6
Prof 8 con asignatura 6 en aula 5.  Costo = 3
Prof 9 con asignatura 8 en aula 6.  Costo = 3


El modelo del problema de asignación 3-dimensional **planar** es el siguiente. Nótese que **no** es un modelo alternativo para el problema de asignación de profesores, asignatura y aulas. Es el modelo de otro problema diferente. Por eso la solución óptima del modelo anterior y del modelo siguiente no coinciden, ni tienen relación alguna. Si tienes curiosidad por conocer el problema asociado a este otro modelo, te sugiero que estudies el concepto de [Cuadrado Latino](https://es.wikipedia.org/wiki/Cuadrado_latino).

In [62]:
solver = pywraplp.Solver('3APplanar', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

x = { (i,j,k) : solver.BoolVar('x[%i,%i,%i]' % (i,j,k)) for i in opciones for j in opciones for k in opciones }
            
solver.Minimize( solver.Sum(cost[i][j][k] * x[i,j,k] for i in opciones for j in opciones for k in opciones) )

[ solver.Add(solver.Sum(x[i,j,k] for k in opciones) == 1) for i in opciones for j in opciones ]
[ solver.Add(solver.Sum(x[i,j,k] for j in opciones) == 1) for i in opciones for k in opciones ]
[ solver.Add(solver.Sum(x[i,j,k] for i in opciones) == 1) for j in opciones for k in opciones ]
    
solver.Solve()

print('Costo total = ', solver.Objective().Value() ," in ", solver.WallTime()/1000, "seconds")
for i in opciones:
    for j in opciones:
        for k in opciones:
            if x[i,j,k].solution_value() > EPS :
                print('Indices: %d  %d  %d.  Costo = %d' % (i,j,k,cost[i][j][k]))

Costo total =  1962.0  in  17.7 seconds
Indices: 0  0  1.  Costo = 9
Indices: 0  1  7.  Costo = 4
Indices: 0  2  0.  Costo = 15
Indices: 0  3  8.  Costo = 6
Indices: 0  4  6.  Costo = 41
Indices: 0  5  2.  Costo = 12
Indices: 0  6  9.  Costo = 8
Indices: 0  7  5.  Costo = 34
Indices: 0  8  4.  Costo = 6
Indices: 0  9  3.  Costo = 11
Indices: 1  0  4.  Costo = 37
Indices: 1  1  8.  Costo = 9
Indices: 1  2  5.  Costo = 52
Indices: 1  3  2.  Costo = 7
Indices: 1  4  3.  Costo = 27
Indices: 1  5  9.  Costo = 15
Indices: 1  6  6.  Costo = 19
Indices: 1  7  0.  Costo = 8
Indices: 1  8  7.  Costo = 41
Indices: 1  9  1.  Costo = 2
Indices: 2  0  3.  Costo = 69
Indices: 2  1  1.  Costo = 13
Indices: 2  2  8.  Costo = 42
Indices: 2  3  9.  Costo = 12
Indices: 2  4  4.  Costo = 3
Indices: 2  5  7.  Costo = 61
Indices: 2  6  0.  Costo = 1
Indices: 2  7  6.  Costo = 1
Indices: 2  8  2.  Costo = 7
Indices: 2  9  5.  Costo = 16
Indices: 3  0  6.  Costo = 26
Indices: 3  1  9.  Costo = 24
Indices: 3  2

Hasta ahora hemos estado usando el software gratuito CBC a través de la herramienta OR-tools de Google. Para resolverlo con Gurobi (recuerda que necesitas licencia sobre el ordenador en el que lo quieres resolver) la sintaxis es:

In [63]:
from gurobipy import *

El modelo del problema de asignación 3-dimensional **planar** es el siguiente. Nótese que **no** es un modelo alternativo para el problema de asignación de profesores, asignatura y aulas. Es el modelo de otro problema diferente. Por eso la solución óptima del modelo anterior y del modelo siguiente no coinciden, ni tienen relación alguna. Si tienes curiosidad por conocer el problema asociado a este otro modelo, te sugiero que estudies el concepto de [Cuadrado Latino](https://es.wikipedia.org/wiki/Cuadrado_latino).

In [64]:
solver = pywraplp.Solver('3APplanar', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

x = { (i,j,k) : solver.BoolVar('x[%i,%i,%i]' % (i,j,k)) for i in opciones for j in opciones for k in opciones }
            
solver.Minimize( solver.Sum(cost[i][j][k] * x[i,j,k] for i in opciones for j in opciones for k in opciones) )

[ solver.Add(solver.Sum(x[i,j,k] for k in opciones) == 1) for i in opciones for j in opciones ]
[ solver.Add(solver.Sum(x[i,j,k] for j in opciones) == 1) for i in opciones for k in opciones ]
[ solver.Add(solver.Sum(x[i,j,k] for i in opciones) == 1) for j in opciones for k in opciones ]
    
solver.Solve()

print('Costo total = ', solver.Objective().Value() ," in ", solver.WallTime()/1000, "seconds")
for i in opciones:
    for j in opciones:
        for k in opciones:
            if x[i,j,k].solution_value() > EPS :
                print('Indices: %d  %d  %d.  Costo = %d' % (i,j,k,cost[i][j][k]))

Costo total =  1962.0  in  14.652 seconds
Indices: 0  0  1.  Costo = 9
Indices: 0  1  7.  Costo = 4
Indices: 0  2  0.  Costo = 15
Indices: 0  3  8.  Costo = 6
Indices: 0  4  6.  Costo = 41
Indices: 0  5  2.  Costo = 12
Indices: 0  6  9.  Costo = 8
Indices: 0  7  5.  Costo = 34
Indices: 0  8  4.  Costo = 6
Indices: 0  9  3.  Costo = 11
Indices: 1  0  4.  Costo = 37
Indices: 1  1  8.  Costo = 9
Indices: 1  2  5.  Costo = 52
Indices: 1  3  2.  Costo = 7
Indices: 1  4  3.  Costo = 27
Indices: 1  5  9.  Costo = 15
Indices: 1  6  6.  Costo = 19
Indices: 1  7  0.  Costo = 8
Indices: 1  8  7.  Costo = 41
Indices: 1  9  1.  Costo = 2
Indices: 2  0  3.  Costo = 69
Indices: 2  1  1.  Costo = 13
Indices: 2  2  8.  Costo = 42
Indices: 2  3  9.  Costo = 12
Indices: 2  4  4.  Costo = 3
Indices: 2  5  7.  Costo = 61
Indices: 2  6  0.  Costo = 1
Indices: 2  7  6.  Costo = 1
Indices: 2  8  2.  Costo = 7
Indices: 2  9  5.  Costo = 16
Indices: 3  0  6.  Costo = 26
Indices: 3  1  9.  Costo = 24
Indices: 3 

Hasta ahora hemos estado usando el software gratuito CBC a través de la herramienta OR-tools de Google. Para resolverlo con Gurobi (recuerda que necesitas licencia sobre el ordenador en el que lo quieres resolver) la sintaxis es:

In [65]:
from gurobipy import *

In [66]:
modelo = Model("AP3axial")
# Resolución del modelo usando Gurobi
# La sintaxis es un poco diferente al anterior, pero se puede entender con facilidad.

x = { (i,j,k) : modelo.addVar(vtype=GRB.BINARY, name="x[%i,%i,%i]" % (i,j,k)) for i in opciones for j in opciones for k in opciones }
            
modelo.modelSense = GRB.MINIMIZE
modelo.setObjective( quicksum(cost[i][j][k] * x[i,j,k] for i in opciones for j in opciones for k in opciones) )
    
[ modelo.addConstr( quicksum(x[i,j,k] for j in opciones for k in opciones) == 1 ) for i in opciones ]
[ modelo.addConstr( quicksum(x[i,j,k] for i in opciones for k in opciones) == 1 ) for j in opciones ]
[ modelo.addConstr( quicksum(x[i,j,k] for i in opciones for j in opciones) == 1 ) for k in opciones ]
                    
modelo.optimize()

# Sin embargo, por la manera en que se imprime el resultado, 
# se puede entender que resuelven el problema usando unas estructuras de datos similares.              
print('Costo total = %g' % modelo.objVal,' in = ',modelo.Runtime, " seconds")
for i in opciones:
    for j in opciones:
        for k in opciones:
            if x[i,j,k].x > EPS:
                print('Indices: %d  %d  %d.  Costo = %d' % (i,j,k,cost[i][j][k]))

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 30 rows, 1000 columns and 3000 nonzeros
Model fingerprint: 0x18768ebb
Variable types: 0 continuous, 1000 integer (1000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 433.0000000
Presolve time: 0.01s
Presolved: 30 rows, 1000 columns, 3000 nonzeros
Variable types: 0 continuous, 1000 integer (1000 binary)

Root relaxation: objective 3.600000e+01, 40 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   36.00000    0   25  433.00000   36.00000

In [67]:
from gurobipy import *

In [69]:
modelo = Model("AP3planar")

# Esta vez resuelve el modelo Planar del problema de asignación.
# Devuelve la misma solución, demostrando que la obtenida por ambos es la óptima.
# Un detalle que he notado, y es que Gurobi resuelve el problema más rápido que la librería de Google.
# Puede ser por las funciones 'quicksum' que tanto se usan.

x = { (i,j,k) : modelo.addVar(vtype=GRB.BINARY, name="x[%i,%i,%i]" % (i,j,k)) for i in opciones for j in opciones for k in opciones }

modelo.modelSense = GRB.MINIMIZE
modelo.setObjective( quicksum(cost[i][j][k] * x[i,j,k] for i in opciones for j in opciones for k in opciones) )
    
[ modelo.addConstr( quicksum(x[i,j,k] for k in opciones) == 1 ) for i in opciones for j in opciones ]
[ modelo.addConstr( quicksum(x[i,j,k] for j in opciones) == 1 ) for i in opciones for k in opciones ]
[ modelo.addConstr( quicksum(x[i,j,k] for i in opciones) == 1 ) for j in opciones for k in opciones ]
                    
modelo.optimize()
                    
print('Costo total = %g' % modelo.objVal,' in = ',modelo.Runtime, " seconds")
for i in opciones:
    for j in opciones:
        for k in opciones:
            if x[i,j,k].x > EPS:
                print('Indices: %d  %d  %d.  Costo = %d' % (i,j,k,cost[i][j][k]))

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 300 rows, 1000 columns and 3000 nonzeros
Model fingerprint: 0x5d8e236e
Variable types: 0 continuous, 1000 integer (1000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 300 rows, 1000 columns, 3000 nonzeros
Variable types: 0 continuous, 1000 integer (1000 binary)

Root relaxation: objective 1.888740e+03, 474 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1888.74016    0  239          - 1888.74016      -     -    0s
H    0     0             

## Conclusión

Como conclusión final comentar que estos problemas de asignación siempre son muy interesantes porque aplacan problemas de la vida real.

Se puede comprobar como el propio código es muy flexible y permite muchas variaciones respecto al modelo, como los equipos o restricciones con datos fijos.

Por otro lado, he digerido un poco más el funcionamiento del Cuadrado Latino, y debo decir que puede tener usos prácticos.